In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from smooth.adam_general.core.adam import ADAM
import numpy as np
import pandas as pd

# Two-Stage Initialization Tests

Two-stage initialization works by:
1. First running a model with `initial="complete"` (full backcasting) to get good starting values
2. Using those values as initial guesses for optimization, allowing parameter refinement

This should produce results that are:
- Different from pure backcasting (since parameters are refined)
- Different from pure optimal (since starting values are better)
- Generally better or similar quality to both methods

### Test 1: Global level ETS(A,N,N) - two-stage vs optimal vs backcasting

In [3]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [4]:
model_optimal = ADAM(model='ANN', lags=[12], initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,N,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with optimal initial:
Parameters: [2.50250224e-02 9.85091808e+01]
Forecast: [101.13396235 101.13396235 101.13396235]


In [5]:
model_backcasting = ADAM(model='ANN', lags=[12], initial='backcasting', n_iterations=2)
model_backcasting.fit(ts_df)
print('ETS(A,N,N) with backcasting (n_iterations=2):')
print('Parameters:', model_backcasting.adam_estimated['B'])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with backcasting (n_iterations=2):
Parameters: [0.]
Forecast: [99.61910829 99.61910829 99.61910829]


In [6]:
model_two_stage = ADAM(model='ANN', lags=[12], initial='complete', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,N,N) with complete initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with complete initialization:
Parameters: [0.]
Forecast: [99.61910829 99.61910829 99.61910829]


In [7]:
model_two_stage = ADAM(model='ANN', lags=[12], initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,N,N) with two-stage initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with two-stage initialization:
Parameters: [ 0.         99.61910829]
Forecast: [99.61910829 99.61910829 99.61910829]


### Test 2: Local trend ETS(A,A,N)

In [8]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [9]:
model_optimal = ADAM(model='AAN', lags=[12], initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,A,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with optimal initial:
Parameters: [1.97972595e-01 1.94752148e-08 9.98087967e+01 5.38301829e-01]
Forecast: [164.80276159 165.3410634  165.87936522]


In [10]:
model_optimal = ADAM(model='AAN', lags=[12], initial='complete')
model_optimal.fit(ts_df)
print('ETS(A,A,N) with complete initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with complete initial:
Parameters: [0.19861903 0.        ]
Forecast: [165.04502127 165.63032178 166.2156223 ]


In [11]:
model_optimal = ADAM(model='AAN', lags=[12], initial='complete')
model_optimal.fit(ts_df)
print('ETS(A,A,N) with complete initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with complete initial:
Parameters: [0.19861903 0.        ]
Forecast: [165.04502127 165.63032178 166.2156223 ]


In [12]:
model = ADAM(model='AAN', lags=[12], initial='two-stage', n_iterations=2)
model.fit(ts_df)
print('ETS(A,A,N) with two-stage')
print('Parameters:', model.adam_estimated['B'])
print('Forecast:', model.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with two-stage
Parameters: [ 4.52236725e-01  0.00000000e+00  6.90832274e+01 -6.30519456e-04]
Forecast: [165.42899646 165.42836594 165.42773542]


### Test 3: Seasonal data ETS(A,A,A)

In [13]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [14]:
model_optimal = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,A,A) with optimal initial:')
print('Alpha, Beta, Gamma:', model_optimal.adam_estimated['B'][:3])
print('Forecast:', model_optimal.   predict(h=12)['mean'].values[:3])

ETS(A,A,A) with optimal initial:
Alpha, Beta, Gamma: [0.07879636 0.01339435 0.00066406]
Forecast: [143.14273609 156.67231457 171.78130318]


In [15]:
model_backcasting = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(A,A,A) with backcasting initial:')
print('Alpha, Beta, Gamma:', model_backcasting.adam_estimated['B'][:3])
print('Forecast:', model_backcasting.   predict(h=12)['mean'].values[:3])

ETS(A,A,A) with backcasting initial:
Alpha, Beta, Gamma: [0.10150039 0.         0.07076862]
Forecast: [143.33415249 156.88769908 172.45763691]


In [16]:
model_complete = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='complete')
model_complete.fit(ts_df)
print('ETS(A,A,A) with complete initial:')
print('Alpha, Beta, Gamma:', model_complete.adam_estimated['B'][:3])
print('Forecast:', model_complete.   predict(h=12)['mean'].values[:3])

ETS(A,A,A) with complete initial:
Alpha, Beta, Gamma: [0.10150039 0.         0.07076862]
Forecast: [143.33415249 156.88769908 172.45763691]


In [17]:
model_two_stage = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,A,A) with two-stage initialization:')
print('Alpha, Beta, Gamma:', model_two_stage.adam_estimated['B'][:3])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,A,A) with two-stage initialization:
Alpha, Beta, Gamma: [0.18929509 0.         0.00040694]
Forecast: [149.26994581 163.02701859 177.79606348]


### Test 4: Damped trend ETS(A,Ad,N)

In [18]:
model_optimal = ADAM(model='AAdN', lags=[12], initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,Ad,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,Ad,N) with optimal initial:
Parameters: [ 9.35411588e-03  9.34792643e-03  9.23445768e-01  2.07115283e+02
 -5.87648565e-01]
Forecast: [168.37759048 168.68823705 168.9751023 ]


In [19]:
model_backcasting = ADAM(model='AAdN', lags=[12], initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(A,Ad,N) with backcasting initial:')
print('Parameters:', model_backcasting.adam_estimated['B'])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(A,Ad,N) with backcasting initial:
Parameters: [0.00290078 0.         1.        ]
Forecast: [167.55482126 167.34762835 167.14043543]


In [20]:
model_two_stage = ADAM(model='AAdN', lags=[12], initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,Ad,N) with two-stage initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,Ad,N) with two-stage initialization:
Parameters: [  0.           0.           0.89011865 176.54989816   0.28538162]
Forecast: [178.8616964 178.8616964 178.8616964]


### Test 5: Multiplicative error ETS(M,N,N)

In [21]:
model_optimal = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='optimal')
model_optimal.fit(ts_df)
print('ETS(M,N,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(M,N,N) with optimal initial:
Parameters: [3.29329547e-02 1.89756969e+02]
Forecast: [174.24319564 174.24319564 174.24319564]


In [22]:
model_backcasting = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(M,N,N) with backcasting initial:')
print('Parameters:', model_backcasting.adam_estimated['B'])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(M,N,N) with backcasting initial:
Parameters: [0.03312988]
Forecast: [174.19715699 174.19715699 174.19715699]


In [23]:
model_two_stage = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(M,N,N) with two-stage initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(M,N,N) with two-stage initialization:
Parameters: [3.29888732e-02 1.05878873e+03]
Forecast: [174.23830382 174.23830382 174.23830382]


### Test 6: Multiplicative seasonal ETS(M,A,M)

In [24]:
model_optimal = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='optimal')
model_optimal.fit(ts_df)
print('ETS(M,A,M) with optimal initial:')
print('Alpha, Beta, Gamma:', model_optimal.adam_estimated['B'][:3])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with optimal initial:
Alpha, Beta, Gamma: [0.10931488 0.00429499 0.00089705]
Forecast: [144.82822464 158.65446106 174.38588348]


In [25]:
model_backcasting = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(M,A,M) with backcasting initial:')
print('Alpha, Beta, Gamma:', model_backcasting.adam_estimated['B'][:3])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with backcasting initial:
Alpha, Beta, Gamma: [0.09784527 0.         0.06255799]
Forecast: [145.35744864 158.11464487 172.63571566]


In [26]:
model_complete = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='complete')
model_complete.fit(ts_df)
print('ETS(M,A,M) with complete initial:')
print('Alpha, Beta, Gamma:', model_complete.adam_estimated['B'][:3])
print('Forecast:', model_complete.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with complete initial:
Alpha, Beta, Gamma: [0.09784527 0.         0.06255799]
Forecast: [145.35744864 158.11464487 172.63571566]


In [27]:
model_two_stage = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(M,A,M) with two-stage initialization:')
print('Alpha, Beta, Gamma:', model_two_stage.adam_estimated['B'][:3])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with two-stage initialization:
Alpha, Beta, Gamma: [2.00006767e-01 0.00000000e+00 6.66215691e-05]
Forecast: [150.21420356 165.01155178 180.80347495]


## Summary

Two-stage initialization successfully:
- Runs a complete backcasting model first
- Extracts parameters (persistence, damping, ARMA)
- Extracts and normalizes initial states
- Uses these as starting values for optimization
- Produces results that combine benefits of both approaches